# sxs_eample.ipynb

This notebook demonstrates how to use the `sxs` python library to interact with the SXS Catalog and its metadata on zenodo. It produces the same output as `GetURLsForCatalogJSON.ipynb` and `ParseCatalogJson.ipynb`, but much faster and more easily.

In [ ]:
import sxs
from sxs import zenodo as zen
import json

This cell sends a single zenodo search query to find all public SXS simulations. It takes minutes, whereas directly sending a query for each piece of metadata as in `GetURLsForCatalogJSON.ipynb` takes closer to an hour.

In [ ]:
%%time
md = zen.api.Records.search(q='communities:sxs AND access_right:open')

The search returns a list of json files. Here loop over each returned result. Make sure that the title of the page ends in an SXS ID by checking that splitting the last word of the title by `:` returns 3 elements. Then, if the type is `BBH`, add the key and metadata to a dictionary.

In [ ]:
catalog_json = {}
for simulation in md:
    sxs_id = simulation['title'].split(' ')[-1]
    if (len(sxs_id.split(':')) == 3):
        if (sxs_id.split(':')[-2] == 'BBH'):
            catalog_json[sxs_id] = simulation

In [ ]:
with open("sxs_catalog_zen.json", 'w') as file:
    file.write(json.dumps(catalog_json))

In [ ]:
print("There are " + str(len(catalog_json.keys())) + " BBH simulations in the catalog.")

In [ ]:
def resolutions_for_simulation(sxs_id):
    resolutions = []
    files = catalog_json[sxs_id]['files']
    for file in files:
        split_filename = file['filename'].split('/')
        if (str(split_filename[-1]) == "Horizons.h5"):
            resolutions.append(int(split_filename[-2].split('Lev')[-1]))
    return sorted(resolutions)

In [ ]:
resolutions_available = {}
for simulation in catalog_json:
    resolutions_available[simulation] = resolutions_for_simulation(simulation)
    
with open("sxs_catalog_zen_resolutions_available.json", 'w') as file:
    file.write(json.dumps(resolutions_available))